In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
import numpy as np
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator

In [6]:
# Initializing the CNN

np.random.seed(1337)
classifier = Sequential()

classifier.add(Convolution2D(32, 3, 3, input_shape = (128, 128, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(16, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# classifier.add(Convolution2D(8, 3, 3, activation = 'relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))



classifier.add(Flatten())

#hidden layer
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.5))

#output layer
classifier.add(Dense(units = 10, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(classifier.summary())
#plot_model(classifier, show_shapes=True, to_file='PlantVillage_CNN.png')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 42, 42, 32)        896       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 21, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 16)          4624      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 3, 3, 16)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 144)               0         
                                                                 
 dense_1 (Dense)             (None, 128)              

In [7]:
#Part 2 - fitting the data set

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(128, 128),
        batch_size=64,
        class_mode='categorical' )
label_map = (training_set.class_indices)

print(label_map)

test_set = test_datagen.flow_from_directory(
        'val',
        target_size=(128, 128),
        batch_size=64,
        class_mode='categorical')


# fitting the model
classifier.fit_generator(
        training_set,
        steps_per_epoch=20,
        epochs=10000,
        validation_data=test_set,
        validation_steps=100)


classifier.save_weights('keras_potato_trained_model_weights.h5')
print('Saved trained model as %s ' % 'keras_potato_trained_model_weights.h5')


Found 10000 images belonging to 10 classes.
{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites Two-spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}
Found 1000 images belonging to 10 classes.


/var/folders/rj/gwmtr5qj72g1yp53gy3xhz9m0000gn/T/ipykernel_29488/626870857.py:27: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/10000
20/20 [==============================] - 19s 920ms/step - loss: 2.3050 - accuracy: 0.1164 - val_loss: 2.2986 - val_accuracy: 0.1250
Epoch 2/10000
20/20 [==============================] - 10s 472ms/step - loss: 2.2946 - accuracy: 0.1195
Epoch 3/10000
20/20 [==============================] - 11s 584ms/step - loss: 2.2884 - accuracy: 0.1273
Epoch 4/10000
20/20 [==============================] - 9s 473ms/step - loss: 2.2650 - accuracy: 0.1445
Epoch 5/10000
20/20 [==============================] - 9s 438ms/step - loss: 2.2224 - accuracy: 0.1969
Epoch 6/10000
20/20 [==============================] - 11s 541ms/step - loss: 2.1360 - accuracy: 0.2266
Epoch 7/10000
20/20 [==============================] - 13s 662ms/step - loss: 2.0050 - accuracy: 0.2977
Epoch 8/10000
20/20 [==============================] - 9s 444ms/step - loss: 1.9029 - accuracy: 0.3289
Epoch 9/10000
20/20 [==============================] - 10s 468ms/step - loss: 1.8315 - accuracy: 0.3750
Epoch 10/10000
20/20 [===

20/20 [==============================] - 11s 554ms/step - loss: 0.6789 - accuracy: 0.7711
Epoch 76/10000
20/20 [==============================] - 11s 537ms/step - loss: 0.7142 - accuracy: 0.7427
Epoch 77/10000
20/20 [==============================] - 10s 490ms/step - loss: 0.6996 - accuracy: 0.7539
Epoch 78/10000
20/20 [==============================] - 15s 774ms/step - loss: 0.7359 - accuracy: 0.7422
Epoch 79/10000
20/20 [==============================] - 11s 567ms/step - loss: 0.6607 - accuracy: 0.7703
Epoch 80/10000
20/20 [==============================] - 9s 437ms/step - loss: 0.6821 - accuracy: 0.7852
Epoch 81/10000
20/20 [==============================] - 14s 717ms/step - loss: 0.6577 - accuracy: 0.7727
Epoch 82/10000
20/20 [==============================] - 15s 728ms/step - loss: 0.6291 - accuracy: 0.7875
Epoch 83/10000
20/20 [==============================] - 18s 908ms/step - loss: 0.6556 - accuracy: 0.7805
Epoch 84/10000
20/20 [==============================] - 21s 1s/step - l

KeyboardInterrupt: 